In [1]:
import pandas as pd
import altair as alt

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [65]:
inf = pd.read_csv("/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/infrastructure/Figure_1__Market_sector_investment_in_infrastructure_increased_by_£2.9_billion_in_2024_compared_with_2023_.csv",
                  skiprows=6)

In [66]:
inf = inf.rename(columns={'Industry group [note 1]': 'Year'})
inf = inf.melt(id_vars=["Year"], var_name="Type", value_name="Amount")

In [70]:
# Rename Type values for better display
inf['Type'] = inf['Type'].replace('Support activities for transportation', 'Transportation')

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_49888/3422535982.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  inf['Type'] = inf['Type'].replace('Support activities for transportation', 'Transportation')


In [71]:
# Convert Type to ordered categorical (reverse alphabetical) and calculate stack positions
type_order = sorted(inf['Type'].unique(), reverse=True)
inf['Type'] = pd.Categorical(inf['Type'], categories=type_order, ordered=True)

# Sort for stacking order
inf_sorted = inf.sort_values(['Year', 'Type'], ascending=[True, False]).copy()

# Calculate the cumulative sum (top of the stack)
inf_sorted['stack_top'] = inf_sorted.groupby('Year')['Amount'].cumsum()

# Calculate the bottom of the stack
inf_sorted['stack_bottom'] = inf_sorted['stack_top'] - inf_sorted['Amount']

# Calculate the midpoint for label placement
inf_sorted['stack_mid'] = ((inf_sorted['stack_top'] - inf_sorted['stack_bottom']) / 2) + inf_sorted['stack_bottom']

# Get the last year for each type and the corresponding stack_mid value
last_points = (
    inf_sorted
    .sort_values('Year')
    .groupby('Type')
    .tail(1)
    .copy()
)

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_49888/509381879.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('Type')


In [129]:
# Manual adjustments for text positioning (modify as needed)
manual_adjustments = {
    # 'Type_name': {'dx': offset_x, 'dy': offset_y}
   'Water supply': {'dx': 20, 'dy': 0},
   'Transportation': {'dx': 20, 'dy': -2.5},
   'Telecommunications': {'dx': 20, 'dy': 0},
   'Sewerage and waste': {'dx': 20, 'dy': 2},
   'Other': {'dx': 20, 'dy': 1},
}

# Apply manual adjustments to the dataframe
last_points_adj = last_points.copy()
last_points_adj['manual_dx'] = 20  # default dx
last_points_adj['manual_dy'] = 0   # default dy

for type_name, adjustments in manual_adjustments.items():
    mask = last_points_adj['Type'] == type_name
    if 'dx' in adjustments:
        last_points_adj.loc[mask, 'manual_dx'] = adjustments['dx']
    if 'dy' in adjustments:
        last_points_adj.loc[mask, 'manual_dy'] = adjustments['dy']

bars = alt.Chart(inf_sorted).mark_bar().encode(
    x=alt.X('Year:N',
            title='Source: ONS - Infrastructure in the UK, investment and net stocks', 
            axis=alt.Axis(labelExpr="datum.value % 2 == 0 ? datum.label : ''", 
                          labelFontSize=14,
                          titleFontSize=12,
                          titleX=175,
                          titleY=30,
                          titleFontStyle='italic')),
    y=alt.Y('Amount:Q', 
            title='Investment',
            axis=alt.Axis(labelExpr="'£' + datum.value / 1000 + ' bn'", 
                          labelFontSize=14, 
                          titleFontSize=14,
                          titleX=-70)),
    color=alt.Color('Type:N', sort=type_order, legend=None),
    order=alt.Order('Type:N', sort='ascending')
)

# Create individual text layers for each unique dx/dy combination
text_layers = []
for _, row in last_points_adj.iterrows():
    single_point = last_points_adj[last_points_adj.index == row.name]
    text_layer = alt.Chart(single_point).mark_text(
        align='left',
        fontWeight='bold',
        fontSize=14,
        dx=int(row['manual_dx']),
        dy=int(row['manual_dy'])
    ).encode(
        x=alt.X('Year:N'),
        y=alt.Y('stack_mid:Q'),
        text='Type:N',
        color=alt.Color('Type:N', legend=None)
    )
    text_layers.append(text_layer)

# Combine all text layers
all_text = alt.layer(*text_layers)

chart = (bars + all_text).properties(width=600, height=400).properties(title={
    "text": "Market sector investment in infrastructure",
    "fontSize": 20, 
})

chart

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_49888/3378124506.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  last_points_adj.loc[mask, 'manual_dy'] = adjustments['dy']


alt.LayerChart(...)

In [130]:
# Save to png
chart.save('Infra_inv.png', scale_factor=2)
chart.save('Infra_inv.json', scale_factor=2)

In [ ]:
# Social media post
# Energy has increased consistently as a proportion of UK market sector investment in infrastructure primarily at the expense of mining and quarrying. This demonstrates the changing economic priorities of the UK economy.